In [1]:
setwd('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1009. NC ENVRIOSCAN/P1009.3. Analyses/P1009.3.3. Data Imputation')
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1009. NC ENVRIOSCAN/P1009.3. Analyses/P1009.3.3. Data Imputation/Output')
cur_date = "020223"

library(readxl)
library(tidyverse)
library(reshape2)
library(missForest)

# reading in files
housing_df = data.frame(read_excel("Input/Housing_Data_013023.xlsx", sheet = 2))

# making some of the cols numeric
housing_df$Median_House_Value = as.numeric(housing_df$Median_House_Value)
housing_df$Median_Gross_Rent = as.numeric(housing_df$Median_Gross_Rent)
housing_df$Median_Gross_Rent_Percentage_of_Household_Income = as.numeric(housing_df$Median_Gross_Rent_Percentage_of_Household_Income)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


In [2]:
head(housing_df)

,GEO_ID,Census_Tract,County,Owner_Occupied_Housing_Units,Owner_Occupied_Housing_Units_Less_than_HPI,Median_House_Value,Median_Gross_Rent,Median_Gross_Rent_Percentage_of_Household_Income
,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,14000US37001020100,201.00,Alamance,853,604,204300,619,25.2
2,14000US37001020200,202.00,Alamance,686,673,79800,759,33.3
3,14000US37001020301,203.01,Alamance,621,621,95900,847,40.0
4,14000US37001020302,203.02,Alamance,935,906,88700,954,19.9
5,14000US37001020400,204.00,Alamance,1475,1433,85700,767,27.3
6,14000US37001020501,205.01,Alamance,950,845,160200,811,21.2


# Variable & GEO Id Filtering

Removing variables and GEO ids that have less than 50% of data prior to imputation

In [3]:
# only keeping variables with at least 50% of data 
variable_presence_df = housing_df %>% 
    pivot_longer(cols = 5:8, names_to = "Variable", values_to = "Value") %>%
    # creating 2 count cols: 1st for non-missing data, 2nd for all data
    mutate(data_count = ifelse(is.na(Value), 0, 1), all_count = 1) %>%
    group_by(Variable) %>%
    summarize(Variable_Presence_Percent = sum(data_count)/ sum(all_count) * 100) %>%
    arrange(Variable_Presence_Percent)

head(variable_presence_df)

Variable,Variable_Presence_Percent
<chr>,<dbl>
Median_Gross_Rent,94.64820
Median_Gross_Rent_Percentage_of_Household_Income,95.84581
Median_House_Value,96.93114
Owner_Occupied_Housing_Units_Less_than_HPI,100.00000


In [4]:
# geo id percentages
geo_id_percent_occurence = apply(housing_df[5:8], 1, function(x) (length(na.omit(x))/length(x)) * 100)
                                 
# creating a df and adding back in geo ids
id_presence_df = data.frame(matrix(ncol = 2, nrow = length(geo_id_percent_occurence)))
colnames(id_presence_df) = c("GEO_ID", "GEO_ID_Presence_Percentage")
id_presence_df$GEO_ID = housing_df$GEO_ID
id_presence_df$GEO_ID_Presence_Percentage = geo_id_percent_occurence
                                 
id_presence_df = id_presence_df %>%
    filter(GEO_ID_Presence_Percentage < 50) %>%
    arrange(GEO_ID_Presence_Percentage)
                                 
head(id_presence_df)

,GEO_ID,GEO_ID_Presence_Percentage
,<chr>,<dbl>
1,14000US37007920302,25
2,14000US37007980000,25
3,14000US37019020312,25
4,14000US37019990100,25
5,14000US37031980100,25
6,14000US37031990100,25


In [5]:
# number of geo ids to be removed
dim(id_presence_df)[1]

[1] 39

In [6]:
# getting 39 geo ids to be removed
removed_geo_ids = id_presence_df$GEO_ID

`%notin%` <- Negate(`%in%`)
filtered_housing_df = housing_df %>%
    filter(GEO_ID %notin% removed_geo_ids)

head(filtered_housing_df)
dim(filtered_housing_df)

,GEO_ID,Census_Tract,County,Owner_Occupied_Housing_Units,Owner_Occupied_Housing_Units_Less_than_HPI,Median_House_Value,Median_Gross_Rent,Median_Gross_Rent_Percentage_of_Household_Income
,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,14000US37001020100,201.00,Alamance,853,604,204300,619,25.2
2,14000US37001020200,202.00,Alamance,686,673,79800,759,33.3
3,14000US37001020301,203.01,Alamance,621,621,95900,847,40.0
4,14000US37001020302,203.02,Alamance,935,906,88700,954,19.9
5,14000US37001020400,204.00,Alamance,1475,1433,85700,767,27.3
6,14000US37001020501,205.01,Alamance,950,845,160200,811,21.2


[1] 2633    8

All variables will be kept, but now we're left with 2,633 geo ids. 

# Random Forest Imputation
Generating missing values using random forest (RF).

In [7]:
# imputing sociodemographic data using RF
random_forest_imputation = function(preimputed_df){

    imputed_RF_object = missForest(as.matrix(preimputed_df[,4:8]))
    imputed_RF_df = imputed_RF_object$ximp
    imputed_values_df = data.frame(cbind(preimputed_df[,1:3], imputed_RF_df)) 

    
    return(imputed_values_df)
}

# calling function
imputed_df = random_forest_imputation(filtered_housing_df)
head(imputed_df)

,GEO_ID,Census_Tract,County,Owner_Occupied_Housing_Units,Owner_Occupied_Housing_Units_Less_than_HPI,Median_House_Value,Median_Gross_Rent,Median_Gross_Rent_Percentage_of_Household_Income
,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,14000US37001020100,201.00,Alamance,853,604,204300,619,25.2
2,14000US37001020200,202.00,Alamance,686,673,79800,759,33.3
3,14000US37001020301,203.01,Alamance,621,621,95900,847,40.0
4,14000US37001020302,203.02,Alamance,935,906,88700,954,19.9
5,14000US37001020400,204.00,Alamance,1475,1433,85700,767,27.3
6,14000US37001020501,205.01,Alamance,950,845,160200,811,21.2


In [8]:
# exporting data
write.csv(imputed_df, paste0(Output,"/", cur_date, "_Imputed_Housing_Data.csv"), row.names = FALSE)